In [16]:
import numpy as np

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다. step 09를 참조하세요.'.format(type(data)))
        
        self.data = data
        self.grad = None    # gradient = 기울기
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        """
        # 재귀 구조
        f = self.creator    # 1. 함수를 가져온다
        if f is not None:
            x = f.input    # 2. 함수의 입력을 가져온다
            x.grad = f.backward(self.grad)    # 3. 함수의 backward method를 호출한다
            x.backward()    # 하나 앞 변수의 backward method를 호출한다
        """
        # 반복 구조 (처리 효율 개선)
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()    # 함수를 가져온다
            x, y = f.input, f.output    # 함수의 입출력을 가져온다
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)
        
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)    # 구체적인 계산은 자식 클래스 forward method에서
        output = Variable(as_array(y))    # 항상 output의 type을 ndarray로 고정
        output.set_creator(self)    # 출력 변수에 창조자 설정
        self.input = input    # 입력 변수 기억
        self.output = output    # 출력 변수 기억
        return output
    
    def forward(self, x):    # 순전파
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
        
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx
    
def square(x):
    f = Square()
    return f(x)
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)



step 01

In [7]:
data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = np.array(2.0)
print(x.data)

1.0
2.0


step 02

In [8]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


step 03

In [9]:
f = Square()
g = Exp()

x = Variable(np.array(0.5))
y = f(x)
z = g(y)
t = f(z)
print(t.data)

1.648721270700128


step 04

In [17]:
def numerical_diff(f, x, eps=1e-4):    # 수치미분_중앙차분
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

TypeError: <class 'numpy.float64'>은(는) 지원하지 않습니다. step 09를 참조하세요.

In [11]:
def f(x):
    f = Square()
    g = Exp()
    return f(g(f(x)))

y = Variable(np.array(0.5))
dy = numerical_diff(f, y)
print(dy)

TypeError: <class 'numpy.float64'>은(는) 지원하지 않습니다.

step 06

In [12]:
A = Square()
B = Exp()
C = Square()

# forward : x -> a -> b -> y
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# backward : dy/dy -> dy/db -> dy/da -> dy/dx
y.grad = np.array(1.0)    # dy/dy = 1
b.grad = C.backward(y.grad)    # dy/db = (dy/db)(dy/dy)
a.grad = B.backward(b.grad)    # dy/da = (db/da)(dy/db)
x.grad = A.backward(a.grad)    # dy/dx = (da/dx)(dy/da)
print(x.grad)

3.297442541400256


step 07

In [13]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드를 거슬러 올라간다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [15]:
# backward : y = C(b) 
y.grad = np.array(1.0)

C = y.creator    # 1. 함수를 가져온다
b = C.input    # 2. 함수의 입력을 가져온다
b.grad = C.backward(y.grad)    # 3. 함수의 backward method를 호출한다.

# backward : b = B(a)
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

# backward : a = A(x)
A = a.creator
x = A.input
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256


In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

step 08

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

step 09

In [ ]:


x = Variable(np.array(0.5))
y = square(exp(square(x)))

# 역전파
y.backward()
print(x.grad)

# g = Variable(1.0)    # NG: 오류 발생!

step 10

In [24]:
# 단위 테스트
import unittest

def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)    # 실제 output
        expected = np.array(4.0)    # 예상 output
        self.assertEqual(y.data, expected)
        
    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)
        
    def test_gradient_check(self):    # 기울기 확인을 위한 테스트
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square,x)
        # np.allclose는 두 파라미터의 값이 가까운지를 판정한다.
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

# 교재 94p unittest.main() 을 Jupyter Notebook에서 하는 방법
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
